In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
file_location = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [6]:
data = requests.get(file_location).text

In [7]:
data_c91 = BeautifulSoup(data, 'lxml')

In [8]:
data_columns = ['Postalcode','Borough','Neighborhood']
toronto_data = pd.DataFrame(columns = data_columns)

In [21]:
data_content = data_c91.find('div', class_='mw-parser-output')
formal_data = data_content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

In [22]:
for tr in formal_data.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto_data = toronto_data.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [27]:
toronto_data = toronto_data[toronto_data.Borough!='Not assigned']
toronto_data = toronto_data[toronto_data.Borough!= 0]
toronto_data.reset_index(drop = True, inplace = True)
j = 0
for j in range(0,toronto_data.shape[0]):
    if toronto_data.iloc[j][2] == 'Not assigned':
        toronto_data.iloc[j][2] = toronto_data.iloc[j][1]
        j = j+1
                                 
df = toronto_data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


Clean data

In [28]:
df = df.dropna()
data_nan = 'Not assigned'
df = df[(df.Postalcode != data_nan ) & (df.Borough != data_nan) & (df.Neighborhood != data_nan)]

In [29]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [31]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [32]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"
